In [2]:
!pip install fuzzywuzzy

In [5]:
import fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import pandas as pd

In [69]:
pd.set_option('max_columns', 150)
pd.set_option('max_rows', 1100)

In [11]:
df = pd.read_csv('cleaned_water_data.csv')

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,5,6,7,9,11,12,16,17,18,19,20,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# need to get a list of the main well types from Brian.  Can create a "choice list" in fuzzy wuzzy and apply those all rows
#top ones I see ['Borehole', 'Protected Borehole', 'Tube Well', 'Protected dug well', 'Pump on hand-dug well','Borehole with submersible pump', 'Stand Post', 'spring river lake stream']

In [19]:
df.water_source.value_counts()

Borehole                                                   19558
Protected Borehole                                         16831
Tube Well                                                  16574
Protected dug well                                         13996
Pump on hand-dug well                                      12173
Unknown                                                    11071
Borehole fitted with submersible pump                       9997
Protected Well (no pump)                                    8314
Dug Well                                                    8260
Manual pump on hand-dug well                                7980
gravity river/stream                                        7691
Standpipe or Tapstand                                       7410
Shallow well                                                6367
Protected Spring                                            5438
Spring                                                      5422
Stand Post               

In [44]:
choices= ['Borehole', 'Protected Borehole', 
          'Tube Well', 'Protected dug well', 'Pump on hand-dug well','Borehole with submersible pump',
          'Stand Post', 'spring river lake stream pond', 'municipal water supply', 'tap', 'rain', 'tank', 
          '__MISSING__']

In [23]:
def fuzzymatch(x, choices, scorer, cutoff):
    return process.extractOne(
    x, choices=choices, scorer=scorer, score_cutoff= cutoff)

In [24]:
uganda = df[(df.country_name == 'Uganda')]

In [37]:
uganda.water_source.fillna('__MISSING__', inplace=True)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [109]:
uganda.water_source.value_counts()
#uganda.loc[:,'water_source']

__MISSING__                                      99313
Borehole                                          4467
Unknown                                           2644
Protected Spring                                  2604
Boreholes                                         1643
Protected Deep Borehole                            766
Unprotected Well                                   349
Protected Shallow Well                             250
Unprotected Spring                                 248
rainwater harvesting                               221
Protected Dug Well                                 198
shallow_well                                       158
Municipal Water Supply                             131
borehole                                           109
Protected Spring Catchment                          73
pond scoop                                          61
Private Tap                                         56
Rainwater Collection                                53
ferro_ceme

In [83]:
uganda['results']= uganda.loc[:,'water_source'].apply(
                                            fuzzymatch,
                                            args=(choices, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [110]:
uganda.results.value_counts()

(__MISSING__, 100)                      99313
(Borehole, 100)                          5366
(Protected Borehole, 72)                 2604
(Borehole, 94)                           1643
(Protected dug well, 88)                  602
(Protected dug well, 100)                 198
(tap, 100)                                138
(municipal water supply, 100)             131
(rain, 100)                                33
(spring river lake stream pond, 100)       26
Name: results, dtype: int64

In [84]:
#uganda.loc[~uganda.results.isin(choices[0:5])]
uganda[['water_source', 'results']].loc[~uganda.water_source.isin(['__MISSING__', 'Boreholes', 'Borehole'])]

,water_source,results
207538,Protected Spring,"(Protected Borehole, 72)"
210710,Protected Spring,"(Protected Borehole, 72)"
212337,Protected Spring,"(Protected Borehole, 72)"
212670,Protected Spring,"(Protected Borehole, 72)"
213196,Unknown,None
213869,Protected Spring,"(Protected Borehole, 72)"
214581,Protected Spring,"(Protected Borehole, 72)"
214663,Protected Spring,"(Protected Borehole, 72)"
214908,Protected Spring,"(Protected Borehole, 72)"
214975,Protected Spring,"(Protected Borehole, 72)"


In [88]:
uganda.results.fillna('bad_match')
uganda.groupby(['water_source', 'results']).count()

,,Row ID,country_name,water_tech,status_id,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality,new_report_date,new_install_year,age_well,age_well_days,status_binary
water_source,results,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Borehole,"(Borehole, 100)",4467,4467,139,4467,0,0,0,207,0,4467,0,0,4467,4467,4467,4467,0,4467,0,0,4467,4467,4467,4467,4467,4467,0,0,0,4467,207,207,4467,4467
Boreholes,"(Borehole, 94)",1643,1643,1582,1643,817,0,0,0,1643,1643,1643,1643,1643,1643,1643,1643,1643,0,0,1643,1643,1643,1643,1643,1643,1643,0,0,0,1643,0,0,1643,1643
Lake,"(spring river lake stream pond, 100)",6,6,6,6,0,2,0,6,0,6,0,0,6,6,6,6,6,0,0,0,6,6,6,6,6,6,0,0,0,6,6,6,6,6
Mechanically Drilled Borehole,"(Borehole, 100)",24,24,24,24,0,4,0,24,0,24,0,0,24,24,24,24,24,0,0,0,24,24,24,24,24,24,0,0,0,24,24,24,24,24
Municipal Water Supply,"(municipal water supply, 100)",131,131,0,131,0,0,0,0,0,131,0,0,131,131,131,131,0,131,0,0,131,131,131,131,131,131,0,0,0,131,0,0,131,131
Private Tap,"(tap, 100)",56,56,0,56,32,56,53,56,23,56,56,56,56,56,56,0,56,0,54,56,56,56,56,56,56,56,0,0,0,56,56,56,56,56
Private tap,"(tap, 100)",24,24,24,24,16,21,0,24,0,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,0,0,0,24,24,24,24,24
Protected Deep Borehole,"(Borehole, 100)",766,766,0,766,600,757,598,760,429,766,766,766,766,766,766,40,766,40,754,766,766,766,766,766,766,766,0,0,0,766,760,760,766,766
Protected Dug Well,"(Protected dug well, 100)",198,198,0,198,0,0,0,0,0,198,0,0,198,198,198,198,0,198,0,0,198,198,198,198,198,198,0,0,0,198,0,0,198,198


In [86]:
uganda.results.fillna('bad_match')
uganda.results.isnull().sum()

3613

In [93]:
choices_tech= ['Borehole', 'Protected Borehole', 
          'Tube Well', 'Protected dug well', 'Pump on hand-dug well','Borehole with submersible pump',
          'Stand Post', 'spring river lake stream pond', 'municipal water supply', 'tap', 'rain', 'tank', 
          '__MISSING__', 'Rainwater Harvest Tank', 'Shallow Well']

In [92]:
uganda.water_tech.value_counts()

Deep borehole                                                                25959
Protected spring                                                             24222
Shallow well                                                                 16717
Rainwater Harvest Tank                                                       15417
Public Stand Post                                                             7611
Yard tap for public use                                                       5721
Manual (hand pump)                                                            1480
Kiosk                                                                          770
Non-Mechanized Borehole                                                        549
Valley Tank                                                                    541
Dam                                                                            317
Protected Shallow Well                                                         242
indi

In [94]:
uganda.water_tech.fillna('__MISSING__', inplace=True)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:3549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [95]:
uganda.water_tech.isnull().sum()

0

In [96]:
uganda['results_tech']= uganda.loc[:,'water_tech'].apply(
                                            fuzzymatch,
                                            args=(choices_tech, 
                                                  fuzz.token_set_ratio, 
                                                 70)
)

/Users/chandlermccann/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [119]:
print("number of uganda water tech categories before fuzzy matching",len(uganda.water_tech.unique()))
print("Number of uganda water tech categories after fuzzy matching", len(uganda.results_tech.unique()))
#fuzzy matching results
uganda.results_tech.value_counts()

number of uganda water tech categories before fuzzy matching 106
Number of uganda water tech categories after fuzzy matching 19


(Borehole, 100)                          26723
(Protected Borehole, 72)                 24230
(Shallow Well, 100)                      17076
(tank, 100)                              15958
(__MISSING__, 100)                       12161
(Stand Post, 100)                         7614
(tap, 100)                                5955
(Pump on hand-dug well, 72)               1480
(Protected dug well, 100)                   68
(Rainwater Harvest Tank, 86)                38
(Pump on hand-dug well, 100)                29
(rain, 100)                                  3
(Borehole with submersible pump, 100)        2
(Protected dug well, 88)                     1
(Borehole, 94)                               1
(spring river lake stream pond, 100)         1
(Protected dug well, 95)                     1
(Borehole, 73)                               1
Name: results_tech, dtype: int64

In [99]:
uganda['fuzzy_tech']=uganda.results_tech.apply(lambda x: [y[0] for y in x])

TypeError: 'int' object is not subscriptable

In [106]:
uganda[['water_tech', 'results_tech']].loc[~uganda.water_tech.isin(['__MISSING__'])]

,water_tech,results_tech
38495,Non-Mechanized Borehole,"(Borehole, 100)"
38496,Non-Mechanized Borehole,"(Borehole, 100)"
38497,Non-Mechanized Borehole,"(Borehole, 100)"
38498,Non-Mechanized Borehole,"(Borehole, 100)"
38499,Non-Mechanized Borehole,"(Borehole, 100)"
38500,Non-Mechanized Borehole,"(Borehole, 100)"
38502,Non-Mechanized Borehole,"(Borehole, 100)"
38503,Non-Mechanized Borehole,"(Borehole, 100)"
38504,Non-Mechanized Borehole,"(Borehole, 100)"
38505,Non-Mechanized Borehole,"(Borehole, 100)"


In [107]:
uganda.groupby(['water_tech', 'results_tech']).count()

,,Row ID,country_name,water_source,status_id,management,pay,installer,install_year,status,source,adm1,adm2,wpdx_id,report_date,country_id,activity_id,data_lnk,orig_lnk,photo_lnk,converted,created,updated,lat_deg,lon_deg,Location,Count,fecal_coliform_presence,fecal_coliform_value,subjective_quality,new_report_date,new_install_year,age_well,age_well_days,status_binary,results
water_tech,results_tech,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
BORE HOLE,"(Borehole, 94)",1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1
Borehole,"(Borehole, 100)",2,2,2,2,0,0,0,2,0,2,2,2,2,2,2,2,0,2,2,0,2,2,2,2,2,2,0,0,0,2,2,2,2,2,2
Borehole or tubewell,"(Borehole, 100)",33,33,33,33,0,0,0,0,33,33,0,0,33,33,33,0,33,0,29,33,33,33,33,33,33,33,0,0,0,33,0,0,33,33,33
DEEP BORE HOLE,"(Borehole, 73)",1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1
DEEP BOREHOLE,"(Borehole, 100)",1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1
Deep borehole,"(Borehole, 100)",25959,25959,25959,25959,25959,16355,0,25040,25956,25959,25959,25959,25959,25959,25959,25959,25959,0,0,25959,25959,25959,25959,25959,25959,25959,0,0,0,25959,25040,25040,25959,25959,25959
Hand Dug Well,"(Pump on hand-dug well, 100)",29,29,29,29,0,0,29,0,0,29,0,0,29,29,29,29,29,0,0,29,29,29,29,29,29,29,0,0,0,29,0,0,29,29,29
Manual (hand pump),"(Pump on hand-dug well, 72)",1480,1480,1480,1480,717,0,0,0,1480,1480,1480,1480,1480,1480,1480,1480,1480,0,0,1480,1480,1480,1480,1480,1480,1480,0,0,0,1480,0,0,1480,1480,1480
Mechanized Borehole,"(Borehole, 100)",25,25,25,25,0,0,25,0,0,25,0,0,25,25,25,25,25,0,0,25,25,25,25,25,25,25,0,0,0,25,0,0,25,25,25
